In [1]:
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### This code will produce arbitrary samples of binary deep eutectic solvents, taking the minimum and maximum mole fractions of each component, number of samples desired to make, and number of trials for the k-means clustering (should be much greater than the number of samples desired to produce). This can easily be modified for ternary deep eutectic solvents.

In [2]:
def Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array([min_QAS, min_HBD])
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array([max_QAS, max_HBD])

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)

